In [1]:
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)




using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
--- startup: 8.724 seconds ---


In [6]:
import re

def clean_gutenberg_text(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 2. Remove page numbers or headers like [Page 1]
    text = re.sub(r'\[Page \d+\]', '', text)

    # 3. Normalize whitespace
    # Remove extra spaces
    text = re.sub(r'[ \t]+', ' ', text)
    # Merge broken lines (assume that a line ending with a lowercase letter continues)
    text = re.sub(r'(?<=[a-z,;])\n(?=[a-zA-Z])', ' ', text)
    # Normalize multiple blank lines to exactly two (paragraph break)
    text = re.sub(r'\n{2,}', '\n\n', text)
    # Strip leading/trailing whitespace
    text = text.strip()

    # 4. Fix Unicode characters (basic replacements)
    text = text.replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")
    text = text.replace('—', '-').replace('–', '-')  # em dash and en dash to hyphen
    text = text.replace('\r', '')  # Remove carriage returns if any
    text = text.replace('_','')

    # 6. Write cleaned text
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)

    print(f"Cleaned text saved to {output_path}")


clean_gutenberg_text(
    input_path='winnie_the_pooh.txt',
    output_path='winnie_the_pooh_cleaned.txt',
)

Cleaned text saved to winnie_the_pooh_cleaned.txt


In [7]:
# Input file to process
input_file="winnie_the_pooh_cleaned.txt"

# Output directory to store resulting files in
output_directory="./output_dir/winnie/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="winnie_the_pooh"

booknlp.process(input_file, output_directory, book_id)


--- spacy: 2.433 seconds ---
--- entities: 16.742 seconds ---
--- quotes: 0.016 seconds ---
--- attribution: 64.589 seconds ---
--- name coref: 0.018 seconds ---
--- coref: 18.497 seconds ---
--- TOTAL (excl. startup): 102.343 seconds ---, 30755 words


In [ ]:
import json
from collections import Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

data=proc("./output_dir/princess/princess.book")
#This will print the first character's mod list. No idea what the i is, but it could be token id 
print(data['characters'][0]['mod'])

[{'w': 'rich', 'i': 576}, {'w': 'old', 'i': 815}, {'w': 'big', 'i': 908}, {'w': 'companion', 'i': 1136}, {'w': 'fun', 'i': 1543}, {'w': 'child', 'i': 1794}, {'w': 'beautiful', 'i': 1805}, {'w': 'child', 'i': 1853}, {'w': 'wolf', 'i': 2312}, {'w': 'fond', 'i': 2433}, {'w': 'intimate', 'i': 3109}, {'w': 'doll', 'i': 3224}, {'w': 'large', 'i': 3229}, {'w': 'mother', 'i': 3394}, {'w': 'thing', 'i': 3675}, {'w': 'pupil', 'i': 4286}, {'w': 'interested', 'i': 4618}, {'w': 'child', 'i': 4739}, {'w': 'person', 'i': 5076}, {'w': 'girl', 'i': 5425}, {'w': 'older', 'i': 5455}, {'w': 'punctilious', 'i': 5458}, {'w': 'baby', 'i': 5561}, {'w': 'determined', 'i': 5760}, {'w': 'fond', 'i': 5842}, {'w': 'boy', 'i': 7007}, {'w': 'pretty', 'i': 7438}, {'w': 'clever', 'i': 7977}, {'w': 'glad', 'i': 9473}, {'w': 'kind', 'i': 9526}, {'w': 'guest', 'i': 9565}, {'w': 'girl', 'i': 9577}, {'w': 'child', 'i': 9589}, {'w': 'child', 'i': 9614}, {'w': 'uncomfortable', 'i': 9670}, {'w': 'unhappy', 'i': 9672}, {'w': '

In [ ]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

for character in data["characters"]:

    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]

    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]

        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=100
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))
        print()

Sara rich
Miss Minchin child
Ermengarde trial
Becky heroine
Lottie quiet
Mr. Carrisford interested
Emily doll
Ram Dass near
Captain Crewe young
Lavinia friends
Mr. Carmichael SURE
Mr. Barrow businessman
Jessie friends
Janet sure
Mariette pleased
Guy Clarence fellow
Melchisedec N/A
Monsieur Dufarge sorry
Donald thing
Madame Pascal glad
Sahib ill
Miss St. John N/A
Mr. Carrrisford desperate
Mrs. Carmichael N/A
Papa soldier
The Last Doll fun
Poor Miss Amelia goose
Nora fond
Missee Sahib N/A
James N/A
Miss Crewe N/A
Mr. St. John N/A
little Miss Legh N/A
Monkey N/A
Lady Meredith N/A
Isobel N/A
Eliza slow
Little Missus N/A
Rosalind Gladys N/A
Mr. Montmorency N/A
Russians able
The Mermaids N/A
the Montmorencys N/A
Veronica Eustacia N/A
the Lascar master
Tom N/A
Ermie N/A
Anne N/A
Messrs. Barrow & Skipworth responsible
Lady Pitkin N/A
the Bastille N/A
Rats N/A
Nature N/A
Miss ' Meliar N/A
Sister N/A
Alfred the Great N/A
Ralph Crewe N/A
Miss Ermengarde N/A
Uncle Tom N/A


In [10]:
import pandas as pd

df_tokens = pd.read_csv("./output_dir/winnie/winnie_the_pooh.tokens", delimiter="\t")
df_entities = pd.read_csv("./output_dir/winnie/winnie_the_pooh.entities", delimiter="\t")

#create dictionary with the shape [sentence_ID][token_ID_within_sentence][word]
sentence_and_token_ID_to_word_dict = {}
for character in data["characters"]:
    character_id=character["id"]
    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
    else:
        max_proper_mention = character_id

    unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
    #print(unique_start_token_ids)
    sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
    sentence_ids = sentence_ids.reset_index(drop=True)

    for index, row in sentence_ids.iterrows():
        sentence_ID = row['sentence_ID']
        token_ID = row['token_ID_within_sentence']
        combined_sentence_and_token_ID = str(sentence_ID) +":"+ str(token_ID)
        sentence_and_token_ID_to_word_dict[combined_sentence_and_token_ID] = max_proper_mention
    

In [36]:
#the number sets which character we are looking at
character = data["characters"][7]
character_id=character["id"]

#get all sentencs where COREF = character_id
unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
print(unique_start_token_ids)
sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
sentence_ids = sentence_ids.reset_index(drop=True)
print(sentence_ids)

1101     8141
1105     8162
1112     8205
1121     8307
1122     8311
        ...  
4129    29144
4133    29165
4141    29200
4150    29283
4231    29764
Name: start_token, Length: 143, dtype: int64
     token_ID_within_document  token_ID_within_sentence  sentence_ID
0                        8141                         9          397
1                        8162                        18          398
2                        8205                         1          401
3                        8307                        22          405
4                        8311                        26          405
..                        ...                       ...          ...
138                     29144                         8         1567
139                     29165                        15         1568
140                     29200                        23         1570
141                     29283                        21         1576
142                     29764             

In [12]:
from allennlp.predictors.predictor import Predictor
SRL_MODEL_PATH = "srl.tar.gz"
predictor_srl = Predictor.from_path(SRL_MODEL_PATH)

In [13]:
import utils
import spacy
nlp = spacy.load("en_core_web_sm")

In [14]:
def check_for_arg(n, arg_name, dict_name, sentence_ID):
    if arg_name in n["tags"]:
        index = n['tags'].index(arg_name)
        if str(sentence_ID)+":"+str(index) in dict_name.keys():
            return dict_name[str(sentence_ID)+":"+str(index)]
        else:
            return None
    else:
        return None
    


In [37]:
columns = ['sentence_ID', 'verb', 'agent', 'patient']
srl_results_df = pd.DataFrame(columns=columns)
protag = 'Owl'

# for each mention in our coreference cluster do the following
for index, row in sentence_ids.iterrows():
    token_ID_within_document = row['token_ID_within_document']
    #word = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'word'].values[0]

    sentence_ID = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'sentence_ID'].values[0]
    words_list = df_tokens.loc[df_tokens['sentence_ID'] == sentence_ID, 'word'].tolist()
    token_ID_within_sentence = row['token_ID_within_sentence']
    try:
        sentence_for_srl = ' '.join(words_list)
    except:
        print("Error: ", words_list)
        continue
    predictions_srl = predictor_srl.predict(sentence_for_srl)

    for n in predictions_srl['verbs']:
        if token_ID_within_sentence < len(n['tags']):
            tag = n['tags'][token_ID_within_sentence]
            if "B-ARG0" in tag:

                word = check_for_arg(n, "B-ARG1", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': [protag], 'patient': [word]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)
            elif "B-ARG1" in tag:
                word = check_for_arg(n, "B-ARG0", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': [word], 'patient': [protag]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)

srl_results_df.to_csv('owl_triples.csv', index=False)
del srl_results_df

In [5]:
import pandas as pd
import os
import glob

# Set the path to your folder containing the CSV files
folder_path = 'SRL_out_winnie'  

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Read and concatenate all CSV files
combined_df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

# Remove duplicate rows
combined_df.drop_duplicates(inplace=True)

# Sort by 'sentence_id'
combined_df.sort_values(by='sentence_ID', inplace=True)

# Save the combined, de-duplicated, and sorted DataFrame to a new CSV
output_path = os.path.join(folder_path, 'combined_sorted.csv')
combined_df.to_csv(output_path, index=False)

print(f"Combined CSV saved to {output_path}")

Combined CSV saved to SRL_out_winnie/combined_sorted.csv


In [7]:
import pandas as pd
import os
import glob
import re

# Set the path to your folder containing the CSV files
folder_path = 'SRL_out_little_princess'  # <-- change this

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# List to collect all processed DataFrames
dataframes = []

for file in csv_files:
    filename = os.path.basename(file)
    
    # Extract protagonist name using regex
    match = re.search(r'_output_(.+?)_little_princess\.csv$', filename)
    if not match:
        print(f"Could not extract protagonist from filename: {filename}")
        continue
    # Replace underscores with spaces and capitalize each word
    protagonist = match.group(1).replace('_', ' ').title()

    # Read CSV
    df = pd.read_csv(file)

    # Replace 'protag' in all columns (or limit to one if needed)
    df = df.replace('protag', protagonist)

    # Add to list
    dataframes.append(df)

# Combine all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

# Remove duplicates
combined_df.drop_duplicates(inplace=True)

# Sort by 'sentence_id'
combined_df.sort_values(by='sentence_ID', inplace=True)

# Save to CSV
output_path = os.path.join(folder_path, 'combined_sorted.csv')
combined_df.to_csv(output_path, index=False)

print(f"Combined CSV saved to {output_path}")

Combined CSV saved to SRL_out_little_princess/combined_sorted.csv
